# My Utilities

In [1]:
#nbx --fname="../src/MyUtils.jl" --jl_module="MyUtils"
using LinearAlgebra

unit_vec(a::Float64) = [cos(a);sin(a)];
LinearAlgebra.angle(x::Vector{Float64}) = atan(x[2],x[1]);
peak_to_peak(xs) = (xs .- minimum(xs))./(maximum(xs) - minimum(xs))

polar(x::Vector{Float64}) = [norm(x);atan(x[2],x[1])];
polar_inv(zs::Vector{Float64}, as::Vector{Float64}) = [[z*cos(a);z*sin(a)] for (z,a) in zip(zs,as)];
polar_inv(r_and_phi::Vector{Float64}) = [r_and_phi[1]*cos(r_and_phi[2]);r_and_phi[1]*sin(r_and_phi[2])]
polar_inv(r::Float64, phi::Float64)   = [r*cos(phi);r*sin(phi)]
polar_inv(z::AbstractArray, a::AbstractArray) = z .* cat(cos.(a), sin.(a), dims=ndims(a)+1);


export unit_vec, polar, angle, peak_to_peak, euclidean, polar_inv

In [2]:
#nbx
"""
Stacks vectors on top of each other (as rows, along dim 1)
"""
stack(xs::AbstractVector) = reduce(vcat, transpose.(xs));
unstack(x::Matrix) = [x[i,:] for i=1:size(x,1)]
"""
Stacks vectors horizontally (along dim 2)
"""
hstack(xs::AbstractVector) = reduce(hcat,xs);

export stack, hstack, unstack

In [3]:
xs = [rand(2) for t=1:4];

size( stack(xs)),
size(hstack(xs))

((4, 2), (2, 4))

In [4]:
#nbx
"""
    rot(hd)

Returns 2D rotation matrix.
"""
rot(hd) = [[cos(hd) -sin(hd)]; [sin(hd) cos(hd)]]

export rot

In [5]:
#nbx
using Colors, Plots
col = palette(:default);

Plots.scatter!(xs::Vector{Vector{Float64}}; args...) = scatter!([x[1] for x in xs], [x[2] for x in xs]; args...)
Plots.plot!(xs::Vector{Vector{Float64}}; args...)    = plot!([x[1] for x in xs], [x[2] for x in xs]; args...)

In [6]:
"""
    griddims = cuda_grid(datadims::Tuple{Vararg{Int}},
                         blockdims::Tuple{Vararg{Int}})

Given data dimensions `datadims` and number of threads
in each dimension `blockdims` returns the respective
grid dimensions `griddims` such that

    griddims[i] = ceil(Int, datadims[i]/blockdims[i])

"""
function cuda_grid(datadims::Tuple{Vararg{Int}}, blockdims::Tuple{Vararg{Int}})
    griddims = ceil.(Int, datadims./blockdims)
    return griddims
end

cuda_grid

In [13]:
#nbx
using Fmt: @f_str, format
function summarize_vars(ex::Expr; fstr=f"{1:<10.10} {2:<}")
    for sx in ex.args
        x = getproperty(Main, sx)
        println(format(fstr, sx, typeof(x)))
    end
end;

In [14]:
x = 1
y = "Hi"
summarize_vars(:(x,y); fstr=f"{1:<10.10} {2:<}")

x          Int64
y          String


# My CUDA Utilities

**Cuda availability**

> Reference: https://cuda.juliagpu.org/stable/installation/conditional/

Chek if we have a GPU that is compatible with CUDA.
In a module one can add the following snippet (`__init__` will then be automatically called)
```julia
const _cuda = Ref(false)
function __init__()
    _cuda[] = CUDA.functional()
end;
```



In [ ]:
#nbx
using CUDA
using BenchmarkTools

const _cuda = Ref(false)
function __init__()
    _cuda[] = CUDA.functional()
end;
DeviceArray(x) = _cuda[] ? CuArray(x) : Array(x);

export _cuda, DeviceArray

In [ ]:
#nbx
"""
    griddims = cuda_grid(datadims::Tuple{Vararg{Int}}, 
                         blockdims::Tuple{Vararg{Int}})

Given data dimensions `datadims` and number of threads 
in each dimension `blockdims` returns the respective 
grid dimensions `griddims` such that

    griddims[i] = ceil(Int, datadims[i]/blockdims[i])

"""
function cuda_grid(datadims::Tuple{Vararg{Int}}, blockdims::Tuple{Vararg{Int}})
    griddims = ceil.(Int, datadims./blockdims)
    return griddims
end
export cuda_grid